## BioPAX abstraction: SIF view

> This code contains SPARQL queries allowing to abstract a BioPAX graph into SIF binary interactions.

> The abstraction rules used here are from PathwayCommons: http://www.pathwaycommons.org/pc2/formats#sif_relations 

> These rules are used in the ChiBE software to perform BioPAX to SIF translation.

#### Import librairies and launch SPARQL endpoint

In [15]:
from SPARQLWrapper import SPARQLWrapper, JSON, CSV, N3, XML, TURTLE
import subprocess
import time
import os
from requests.utils import requote_uri
from urllib.parse import quote
import re
import rdflib
import pandas as pd

In [16]:
endpoint_reactome = "http://localhost:3030/reactome"
rdfFormat = "turtle"
current_directory = os.getcwd()
BioPAX_Ontology_file_path = os.path.join(current_directory, '../..', 'ReactomeData', 'biopax-level3.owl')
ReactomeBioPAX_file_path = os.path.join(current_directory, '../..', 'ReactomeData', 'Homo_sapiens_v93.owl')

In [17]:
def extract_prefix_mappings(prefixes_string):
    """
    Extract prefix mappings from the SPARQL prefixes string.
    
    Parameters:
    prefixes_string (str): String containing PREFIX declarations
    
    Returns:
    dict: Mapping of full URIs to their prefixes
    """
    # Extract prefix declarations using regex
    prefix_pattern = re.compile(r'PREFIX\s+(\w+):\s*<([^>]+)>', re.IGNORECASE)
    return {uri: prefix for prefix, uri in prefix_pattern.findall(prefixes_string)}

def convert_to_prefixed_uri(uri_string, prefix_mappings):
    """
    Convert a full URI to prefixed format.
    
    Parameters:
    uri_string (str): Full URI string
    prefix_mappings (dict): Mapping of URIs to prefixes
    
    Returns:
    str: URI in prefixed format (e.g., 'reactome:Protein')
    """
    for uri_base, prefix in prefix_mappings.items():
        if uri_string.startswith(uri_base):
            local_part = uri_string[len(uri_base):]
            return f"{prefix}:{local_part}"
    return uri_string  # Return original if no prefix matches

def save_for_cytoscape(sparql, prefixes_string, output_file, format='csv', separator=','):
    """
    Save SPARQL CONSTRUCT results in a format compatible with Cytoscape,
    using prefix notation for URIs.
    
    Parameters:
    sparql (SPARQLWrapper): Configured SPARQLWrapper instance with query
    prefixes_string (str): String containing PREFIX declarations
    output_file (str): Path to save the output file
    format (str): Output format ('csv' or 'tsv')
    separator (str): Column separator (',' for CSV, '\t' for TSV)
    """
    # Extract prefix mappings
    prefix_mappings = extract_prefix_mappings(prefixes_string)
    
    # Get the results as an RDF graph
    sparql.setReturnFormat(TURTLE)
    results = sparql.queryAndConvert()
    
    # Create an RDFlib graph
    g = rdflib.Graph()
    if isinstance(results, bytes):
        g.parse(data=results.decode('utf-8'), format='turtle')
    else:
        g.parse(data=results, format='turtle')
    
    # Convert triples to a list of dictionaries with prefixed URIs
    triples_data = []
    for s, p, o in g:# Extraction of "Signaling by EGFR" (R-HSA-177929) pathway from Reactome BioPAX export v65
        # Convert each URI to prefixed format
        subject = convert_to_prefixed_uri(str(s), prefix_mappings)
        predicate = convert_to_prefixed_uri(str(p), prefix_mappings)
        object_ = convert_to_prefixed_uri(str(o), prefix_mappings)
        
        triples_data.append({
            'Source': subject,
            'Interaction': predicate,
            'Target': object_
        })
    
    # Convert to DataFrame for easy CSV/TSV export
    df = pd.DataFrame(triples_data)
    
    # Save to file
    if format == 'csv':
        df.to_csv(output_file, index=False, sep=',')
    else:  # tsv
        df.to_csv(output_file, index=False, sep='\t')
    
    print(f"Saved {len(triples_data)} interactions to {output_file}")
    return df

def preview_network_data(df, n=5):
    """
    Preview the network data before importing into Cytoscape.
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing the network data
    n (int): Number of rows to preview
    """
    print(f"\nPreview of network data ({len(df)} total interactions):")
    print(f"\nFirst {n} interactions:")
    print(df.head(n))
    
    # Print some basic network statistics
    unique_nodes = set(df['Source'].unique()) | set(df['Target'].unique())
    print(f"\nNetwork statistics:")
    print(f"Number of unique nodes: {len(unique_nodes)}")
    print(f"Number of interactions: {len(df)}")
    print(f"Unique interaction types:")
    for interaction in sorted(df['Interaction'].unique()):
        print(f"  - {interaction}")

In [18]:
reactomeVersion = 93
prefixes = f"""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX chebi: <http://purl.obolibrary.org/obo/chebi/>
PREFIX chebidb: <http://purl.obolibrary.org/obo/CHEBI_>
PREFIX chebirel: <http://purl.obolibrary.org/obo/CHEBI#>
PREFIX oboInOwl: <http://www.geneontology.org/formats/oboInOwl#>
PREFIX bp3: <http://www.biopax.org/release/biopax-level3.owl#>
PREFIX reactome: <http://www.reactome.org/biopax/{reactomeVersion}/48887#>
PREFIX abstraction:<http://abstraction/#>
"""

biopaxURI = "http://www.biopax.org/release/biopax-level3.owl#"

In [19]:
command = [
    '/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',
    '--file', ReactomeBioPAX_file_path,
    '--file', BioPAX_Ontology_file_path,
    '/reactome']

process = subprocess.Popen(command)
time.sleep(60)

11:33:12 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2025/BioPAXAbstraction/01_SIFAbstraction/Scripts/../../ReactomeData/Homo_sapiens_v93.owl
11:33:13 WARN  riot            :: [line: 62765, col: 34] {W137} Input is large. Switching off checking for illegal reuse of rdf:ID's.
11:33:34 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2025/BioPAXAbstraction/01_SIFAbstraction/Scripts/../../ReactomeData/biopax-level3.owl
11:33:34 INFO  Server          :: Running in read-only mode for /reactome
11:33:34 INFO  Server          :: Apache Jena Fuseki 4.9.0
11:33:34 INFO  Config          :: FUSEKI_HOME=/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0
11:33:34 INFO  Config          :: FUSEKI_BASE=/home/cbeust/Projects/2025/BioPAXAbstraction/01_SIFAbstraction/Scripts/run
11:33:34 INFO  Config          :: Shiro file: file:///home/cbeust/Projects/2025/BioPAXAbstraction/01_SIFAbstraction/Scripts/run/shiro.ini
11:33:34 INFO  Serve

## Translation of SIF abstraction rules in SPARQL queries

#### 1 - Controls State Change Of
First protein controls a reaction that changes the state of the second protein

In [7]:
start_time = time.time()
# TODO : pass to entity reference ??? 
query_controls_state_change_of = """ 
CONSTRUCT {
  ?enzymeProt abstraction:ControlsStateChangeOf ?protLeft 
}
WHERE {
  ?reaction rdf:type/(rdfs:subClassOf*) bp3:BiochemicalReaction .
  
  ?catalysis bp3:controlled ?reaction .
  ?catalysis (rdf:type/rdfs:subClassOf*) bp3:Control .
  ?catalysis bp3:controller ?enzyme .
  ?enzyme ((bp3:component|bp3:memberPhysicalEntity)*) ?enzymeProt .
  ?enzymeProt rdf:type bp3:Protein .
  
  ?reaction bp3:left/((bp3:component|bp3:memberPhysicalEntity)*) ?protLeft .
  ?protLeft rdf:type bp3:Protein .
  ?protLeft bp3:entityReference ?protRef .
  
  ?reaction bp3:right/((bp3:component|bp3:memberPhysicalEntity)*) ?protRight .
  ?protRight rdf:type bp3:Protein .
  ?protRight bp3:entityReference ?protRef .
 
  ?protRight bp3:feature ?feature .
  ?feature (rdf:type/rdfs:subClassOf*) bp3:EntityFeature .
  FILTER (?enzymeProt != ?protLeft)
}
"""


# execute SPARQL query
sparql = SPARQLWrapper(endpoint_reactome)
sparql.setQuery(prefixes+query_controls_state_change_of)
df = save_for_cytoscape(sparql, prefixes, "../Results/ReactomeHomoSapiens93/01-ControlsStateChangeOf.csv", format='csv')
preview_network_data(df)
print("--- %s seconds ---" % (time.time() - start_time))

10:14:29 INFO  Fuseki          :: [1] GET http://localhost:3030/reactome?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0APREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/93/48887%23%3E%0APREFIX+abstraction%3A%3Chttp%3A//abstraction/%23%3E%0A+%0ACONSTRUCT+%7B%0A++%3FenzymeProt+abstraction%3AControlsStateChangeOf+%3FprotL

Saved 504851 interactions to ../Results/ReactomeHomoSapiens93/01-ControlsStateChangeOf.csv

Preview of network data (504851 total interactions):

First 5 interactions:
                  Source                        Interaction  \
0   reactome:Protein5017  abstraction:ControlsStateChangeOf   
1  reactome:Protein16843  abstraction:ControlsStateChangeOf   
2   reactome:Protein2002  abstraction:ControlsStateChangeOf   
3   reactome:Protein6102  abstraction:ControlsStateChangeOf   
4   reactome:Protein5877  abstraction:ControlsStateChangeOf   

                  Target  
0   reactome:Protein4967  
1  reactome:Protein16896  
2    reactome:Protein108  
3   reactome:Protein6143  
4   reactome:Protein5878  

Network statistics:
Number of unique nodes: 13534
Number of interactions: 504851
Unique interaction types:
  - abstraction:ControlsStateChangeOf
--- 325.8826804161072 seconds ---


#### 2 - Controls Transport Of

First protein controls a reaction that changes the cellular location of the second protein

In [10]:
start_time = time.time()
query_transport_of = """
CONSTRUCT {
  ?enzymeProt abstraction:ControlsTransportOf ?protLeft
}
WHERE {
  ?reaction rdf:type/(rdfs:subClassOf*) bp3:BiochemicalReaction .

  ?reaction bp3:left/((bp3:component|bp3:memberPhysicalEntity)*) ?protLeft .
  ?protLeft rdf:type bp3:Protein .
  ?protLeft bp3:entityReference ?protRef .
  
  ?reaction bp3:right/((bp3:component|bp3:memberPhysicalEntity)*) ?protRight .
  ?protRight rdf:type bp3:Protein .
  ?protRight bp3:entityReference ?protRef .

  # Define cellular locations for Protein 1 and Protein 2
  ?protLeft bp3:cellularLocation ?cellularLocVocab1 .
  ?protRight bp3:cellularLocation ?cellularLocVocab2 .
  
  ?catalysis bp3:controlled ?reaction .
  ?catalysis (rdf:type/rdfs:subClassOf*) bp3:Control .
  ?catalysis bp3:controller ?enzyme .
  ?enzyme ((bp3:component|bp3:memberPhysicalEntity)*) ?enzymeProt .
  ?enzymeProt rdf:type bp3:Protein .

  # Ensure cellular locations of protein 1 and protein 2 are different + ensure that they have the same reference
  FILTER (?cellularLocVocab1 != ?cellularLocVocab2)
  FILTER (?enzymeProt != ?protLeft)
}

"""

sparql = SPARQLWrapper(endpoint_reactome)
sparql.setQuery(prefixes+query_transport_of)
df = save_for_cytoscape(sparql, prefixes, "../Results/ReactomeHomoSapiens93/02-ControlsTransportOf.csv", format='csv')
preview_network_data(df)
print("--- %s seconds ---" % (time.time() - start_time))

10:35:20 INFO  Fuseki          :: [4] GET http://localhost:3030/reactome?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0APREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/93/48887%23%3E%0APREFIX+abstraction%3A%3Chttp%3A//abstraction/%23%3E%0A%0ACONSTRUCT+%7B%0A++%3FenzymeProt+abstraction%3AControlsTransportOf+%3FprotLeft

Saved 32687 interactions to ../Results/ReactomeHomoSapiens93/02-ControlsTransportOf.csv

Preview of network data (32687 total interactions):

First 5 interactions:
                  Source                      Interaction  \
0   reactome:Protein2220  abstraction:ControlsTransportOf   
1  reactome:Protein15427  abstraction:ControlsTransportOf   
2   reactome:Protein1858  abstraction:ControlsTransportOf   
3  reactome:Protein15441  abstraction:ControlsTransportOf   
4   reactome:Protein1867  abstraction:ControlsTransportOf   

                  Target  
0   reactome:Protein2193  
1  reactome:Protein26267  
2  reactome:Protein10525  
3  reactome:Protein26275  
4   reactome:Protein9074  

Network statistics:
Number of unique nodes: 3113
Number of interactions: 32687
Unique interaction types:
  - abstraction:ControlsTransportOf
--- 12.157529592514038 seconds ---


#### 3 - Controls Phosphorylation Of

First protein controls a reaction that changes the phosphorylation status of the second protein

Reaction test Reactome : R-HSA-388831, Phosphorylation of CD28

In [11]:
start_time = time.time()
query_controls_phosphorylation_of = """
CONSTRUCT {
	?enzymeProt abstraction:ControlsPhosphorylationOf ?protLeft
	}
WHERE {
  ?reaction rdf:type/(rdfs:subClassOf*) bp3:BiochemicalReaction .
  
  ?catalysis bp3:controlled ?reaction .
  ?catalysis (rdf:type/rdfs:subClassOf*) bp3:Control .
  ?catalysis bp3:controller ?enzyme .
  ?enzyme ((bp3:component|bp3:memberPhysicalEntity)*) ?enzymeProt .
  ?enzymeProt rdf:type bp3:Protein .
  
  ?reaction bp3:left/((bp3:component|bp3:memberPhysicalEntity)*) ?protLeft .
  ?protLeft rdf:type bp3:Protein .
  ?protLeft bp3:entityReference ?protLeftRef .
  
  ?reaction bp3:right/((bp3:component|bp3:memberPhysicalEntity)*) ?protRight .
  ?protRight rdf:type bp3:Protein .
  ?protRight bp3:entityReference ?protRightRef .

  # Complete phosphorylation check
  ?protRight bp3:feature ?feature .
  ?feature rdf:type bp3:ModificationFeature .
  ?feature bp3:modificationType ?modificationType .
  ?modificationType rdf:type bp3:SequenceModificationVocabulary .
  ?modificationType bp3:term ?modificationTerm .
  FILTER(CONTAINS(LCASE(?modificationTerm), "phospho"))
 
  # Ensure left and right proteins are the same (unphosphorylated vs phosphorylated)
  FILTER(?protLeftRef = ?protRightRef)
}
"""

sparql = SPARQLWrapper(endpoint_reactome)
sparql.setQuery(prefixes+query_controls_phosphorylation_of)
df = save_for_cytoscape(sparql, prefixes, "../Results/ReactomeHomoSapiens93/03-ControlsPhosphorylationOf.csv", format='csv')
preview_network_data(df)
print("--- %s seconds ---" % (time.time() - start_time))

10:39:53 INFO  Fuseki          :: [5] GET http://localhost:3030/reactome?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0APREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/93/48887%23%3E%0APREFIX+abstraction%3A%3Chttp%3A//abstraction/%23%3E%0A%0ACONSTRUCT+%7B%0A%09%3FenzymeProt+abstraction%3AControlsPhosphorylationOf+%3Fp

Saved 75758 interactions to ../Results/ReactomeHomoSapiens93/03-ControlsPhosphorylationOf.csv

Preview of network data (75758 total interactions):

First 5 interactions:
                  Source                            Interaction  \
0   reactome:Protein6306  abstraction:ControlsPhosphorylationOf   
1  reactome:Protein26260  abstraction:ControlsPhosphorylationOf   
2  reactome:Protein23252  abstraction:ControlsPhosphorylationOf   
3   reactome:Protein3532  abstraction:ControlsPhosphorylationOf   
4  reactome:Protein25506  abstraction:ControlsPhosphorylationOf   

                  Target  
0  reactome:Protein26319  
1  reactome:Protein26273  
2  reactome:Protein23223  
3   reactome:Protein7335  
4  reactome:Protein25503  

Network statistics:
Number of unique nodes: 5248
Number of interactions: 75758
Unique interaction types:
  - abstraction:ControlsPhosphorylationOf
--- 487.44821429252625 seconds ---


#### 4 - Controls Expression Of
First protein controls a conversion or a template reaction that changes expression of the second protein

In [12]:
start_time = time.time()
query_controls_expression_of = """ 
CONSTRUCT {
  ?enzymeProt abstraction:ControlsExpressionOf ?productProt
}
WHERE {
  # core template reaction pattern
  ?tr rdf:type bp3:TemplateReaction .
  ?tr bp3:product ?product .
  ?product ((bp3:component|bp3:memberPhysicalEntity)*) ?productProt .
  ?productProt rdf:type bp3:Protein .
  
  ?catalysis bp3:controlled ?tr .
  ?catalysis rdf:type bp3:TemplateReactionRegulation .
  ?catalysis bp3:controller ?enzyme .
  ?enzyme ((bp3:component|bp3:memberPhysicalEntity)*) ?enzymeProt .
  ?enzymeProt rdf:type bp3:Protein .
}
"""

sparql = SPARQLWrapper(endpoint_reactome)
sparql.setQuery(prefixes+query_controls_expression_of)
df = save_for_cytoscape(sparql, prefixes, "../Results/ReactomeHomoSapiens93/04-ControlsExpressionOf.csv", format='csv')
preview_network_data(df)
print("--- %s seconds ---" % (time.time() - start_time))

Saved 1 interactions to ../Results/ReactomeHomoSapiens93/04-ControlsExpressionOf.csv

Preview of network data (1 total interactions):

First 5 interactions:
                 Source                       Interaction  \
0  reactome:Protein9323  abstraction:ControlsExpressionOf   

                 Target  
0  reactome:Protein6691  

Network statistics:
Number of unique nodes: 2
Number of interactions: 1
Unique interaction types:
  - abstraction:ControlsExpressionOf
--- 0.03948354721069336 seconds ---


10:48:09 INFO  Fuseki          :: [6] GET http://localhost:3030/reactome?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0APREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/93/48887%23%3E%0APREFIX+abstraction%3A%3Chttp%3A//abstraction/%23%3E%0A+%0ACONSTRUCT+%7B%0A++%3FenzymeProt+abstraction%3AControlsExpressionOf+%3Fproduc

#### 5 - Catalysis Precedes
Equivalent of sequential_catalysis from Paxtools SIF rules
First protein controls a conversion whose input molecule is input to another reaction controlled by a second protein 

Reaction test Reactome : R-HSA-2142688, Synthesis of 5-eicosatetranoic acids

In [ ]:
start_time = time.time()
query_catalysis_precedes = """ 
CONSTRUCT {
   ?enzymeProtA abstraction:CatalysisPrecedes ?enzymeProtB
}
WHERE {
  ?reaction1 rdf:type/(rdfs:subClassOf*) bp3:Conversion .
  ?reaction1 bp3:xref [
    bp3:db "Reactome" ;
    bp3:id ?reaction1ID
  ] .
  
  ?catalysis1 bp3:controlled ?reaction1 .
  ?catalysis1 (rdf:type/rdfs:subClassOf*) bp3:Control .
  ?catalysis1 bp3:controller ?enzymeA .
  ?enzymeA ((bp3:component|bp3:memberPhysicalEntity)*) ?enzymeProtA .
  ?enzymeProtA rdf:type bp3:Protein .
  
  ?reaction1 bp3:right/((bp3:component|bp3:memberPhysicalEntity)*) ?connectingMolecule .
  ?reaction2 bp3:left/((bp3:component|bp3:memberPhysicalEntity)*) ?connectingMolecule .
  ?connectingMolecule rdf:type bp3:SmallMolecule .
  
  ?reaction2 rdf:type/(rdfs:subClassOf*) bp3:Conversion .
  ?reaction2 bp3:xref [
    bp3:db "Reactome" ;
    bp3:id ?reaction2ID
  ] .
  
  ?catalysis2 bp3:controlled ?reaction2 .
  ?catalysis2 (rdf:type/rdfs:subClassOf*) bp3:Control .
  ?catalysis2 bp3:controller ?enzymeB .
  ?enzymeB ((bp3:component|bp3:memberPhysicalEntity)*) ?enzymeProtB .
  ?enzymeProtB rdf:type bp3:Protein .

  FILTER (?enzymeA != ?enzymeB)
}

"""

query_catalysis_precedes = """ 
CONSTRUCT {
   ?enzymeProt1 abstraction:CatalysisPrecedes ?enzymeProt2
}
WHERE {
  ?reaction1 rdf:type/(rdfs:subClassOf*) bp3:BiochemicalReaction .

  ?reaction1 bp3:left/((bp3:component|bp3:memberPhysicalEntity)*) ?protLeft1 .
  ?protLeft1 rdf:type bp3:SmallMolecule .
  ?reaction1 bp3:right/((bp3:component|bp3:memberPhysicalEntity)*) ?protRight1 .
  ?protRight1 rdf:type bp3:SmallMolecule .
  
  ?catalysis1 bp3:controlled ?reaction1 .
  ?catalysis1 rdf:type bp3:Catalysis .
  ?catalysis1 bp3:controller ?enzyme1 .
  ?enzyme1 ((bp3:component|bp3:memberPhysicalEntity)*) ?enzymeProt1 .
  ?enzymeProt1 rdf:type bp3:Protein .
  
  ?reaction2 rdf:type/(rdfs:subClassOf*) bp3:BiochemicalReaction .

  ?reaction2 bp3:left/((bp3:component|bp3:memberPhysicalEntity)*) ?protLeft2 .
  ?protLeft2 rdf:type bp3:SmallMolecule .
  ?reaction2 bp3:right/((bp3:component|bp3:memberPhysicalEntity)*) ?protRight2 .
  ?protRight2 rdf:type bp3:SmallMolecule .
  
  ?catalysis2 bp3:controlled ?reaction2 .
  ?catalysis2 rdf:type bp3:Catalysis .
  ?catalysis2 bp3:controller ?enzyme1 .
  ?enzyme2 ((bp3:component|bp3:memberPhysicalEntity)*) ?enzymeProt2 .
  ?enzymeProt2 rdf:type bp3:Protein .

  # Ensure the controllers of the two reactions are not the same
  FILTER (?enzymeProt1 != ?enzymeProt2)
}
"""

sparql = SPARQLWrapper(endpoint_reactome)
sparql.setQuery(prefixes+query_catalysis_precedes)
df = save_for_cytoscape(sparql, prefixes, "../Results/ReactomeHomoSapiensv92/05-CatalysisPrecedes.csv", format='csv')
preview_network_data(df)
print("--- %s seconds ---" % (time.time() - start_time))

11:46:26 INFO  Fuseki          :: [5] GET http://localhost:3030/reactome?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0APREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/91/48887%23%3E%0APREFIX+abstraction%3A%3Chttp%3A//abstraction/%23%3E%0A+%0ACONSTRUCT+%7B%0A+++%3FenzymeProtA+abstraction%3ACatalysisPrecedes+%3FenzymeP

KeyboardInterrupt: 

#### 6 - In Complex With
Equivalent component of Paxtools SIF rules
Proteins are members of the same complex

Reaction test Reactome : R-HSA-2187368,  STUB1 (CHIP) ubiquitinates SMAD3 

In [13]:
start_time = time.time()
query_in_complex_with = """ 
CONSTRUCT {
  ?prot1 abstraction:InComplexWith ?prot2
}
WHERE {
  # Case 1: Complex with several different proteins
  {
	?complex rdf:type/rdfs:subClassOf* bp3:Complex .
	?complex ((bp3:component|bp3:memberPhysicalEntity)*) ?prot1 .
	?complex ((bp3:component|bp3:memberPhysicalEntity)*) ?prot2 .
	?prot1 rdf:type bp3:Protein .
	?prot2 rdf:type bp3:Protein .
    
	# Avoid pairing proteins from the same collection
	FILTER (?prot1 != ?prot2 || NOT EXISTS { ?prot1 bp3:memberPhysicalEntity+ ?x . ?prot2 bp3:memberPhysicalEntity+ ?y })
    
	# Avoid self-pairing and duplicate pairs
	FILTER (STR(?prot1) < STR(?prot2))
  }
 
  UNION
 
  # Case 2: Same protein with stoichiometry > 1
  {
	?complex rdf:type/rdfs:subClassOf* bp3:Complex .
	# Check for stoichiometry > 1
	?complex bp3:componentStoichiometry ?stoich .
	?stoich bp3:physicalEntity ?entity1 .
	?stoich bp3:stoichiometricCoefficient ?coeff .
	FILTER (?coeff > 1)
	?prot1 rdf:type bp3:Protein .
    
    ?entity1 ((bp3:component|bp3:memberPhysicalEntity)*) ?prot1 .
	?prot1 rdf:type bp3:Protein .
    
	# For self-pairing, use the same entity
	BIND (?prot1 AS ?prot2)
  }
}
"""

sparql = SPARQLWrapper(endpoint_reactome)
sparql.setQuery(prefixes+query_in_complex_with)
df = save_for_cytoscape(sparql, prefixes, "../Results/ReactomeHomoSapiens93/06-InComplexWith.csv", format='csv')
preview_network_data(df)
print("--- %s seconds ---" % (time.time() - start_time))

10:53:31 INFO  Fuseki          :: [7] GET http://localhost:3030/reactome?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0APREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/93/48887%23%3E%0APREFIX+abstraction%3A%3Chttp%3A//abstraction/%23%3E%0A+%0ACONSTRUCT+%7B%0A++%3Fprot1+abstraction%3AInComplexWith+%3Fprot2%0A%7D%0AWHER

Saved 731648 interactions to ../Results/ReactomeHomoSapiens93/06-InComplexWith.csv

Preview of network data (731648 total interactions):

First 5 interactions:
                  Source                Interaction                 Target
0  reactome:Protein10357  abstraction:InComplexWith    reactome:Protein984
1   reactome:Protein3515  abstraction:InComplexWith   reactome:Protein5676
2  reactome:Protein23018  abstraction:InComplexWith  reactome:Protein23032
3  reactome:Protein26987  abstraction:InComplexWith  reactome:Protein26987
4  reactome:Protein16991  abstraction:InComplexWith  reactome:Protein17116

Network statistics:
Number of unique nodes: 19754
Number of interactions: 731648
Unique interaction types:
  - abstraction:InComplexWith
--- 443.5459704399109 seconds ---


#### 7 - Interacts With 
Proteins are participants of the same MolecularInteraction

Problem : no instance of bp3:MolecularInteraction in Reactome

In [15]:
start_time = time.time()
query_interacts_with = """  
CONSTRUCT {
  ?Prot1 abstraction:InteractsWith ?Prot2
}
WHERE {
  # Core molecular interaction pattern
  ?MolecularInteraction (rdf:type/rdfs:subClassOf*) bp3:MolecularInteraction .
  ?MolecularInteraction bp3:participant ?Participant1 .
  ?MolecularInteraction bp3:participant ?Participant2 .
  
  ?Participant1 ((bp3:component|bp3:memberPhysicalEntity)*) ?Prot1 .
  ?Prot1 rdf:type bp3:Protein .
  ?Participant2 ((bp3:component|bp3:memberPhysicalEntity)*) ?Prot2 .
  ?Prot2 rdf:type bp3:Protein .
  
  FILTER (STR(?Prot1) < STR(?Prot2))
}
"""

sparql = SPARQLWrapper(endpoint_reactome)
sparql.setQuery(prefixes+query_interacts_with)
df = save_for_cytoscape(sparql, prefixes, "../Results/ReactomeHomoSapiens93/07-InteractsWith.csv", format='csv')
preview_network_data(df)
print("--- %s seconds ---" % (time.time() - start_time))

11:11:53 INFO  Fuseki          :: [9] GET http://localhost:3030/reactome?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0APREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/93/48887%23%3E%0APREFIX+abstraction%3A%3Chttp%3A//abstraction/%23%3E%0A++%0ACONSTRUCT+%7B%0A++%3FProt1+abstraction%3AInteractsWith+%3FProt2%0A%7D%0AWHE

Saved 0 interactions to ../Results/ReactomeHomoSapiens93/07-InteractsWith.csv

Preview of network data (0 total interactions):

First 5 interactions:
Empty DataFrame
Columns: []
Index: []


KeyError: 'Source'

#### 8 - Neighbor Of 
Proteins are participants or controlers of the same interaction

Reaction test Reactome : R-HSA-170844, Latent TGF-beta-1 is cleaved by FURIN

In [20]:
start_time = time.time()
query_neighbor_of = """ 
CONSTRUCT {
   ?protein1 abstraction:NeighborOf ?protein2
}
WHERE {
  ?reaction rdf:type/(rdfs:subClassOf*) bp3:Interaction .
  
  ?reaction ((^bp3:controlled/bp3:controller)|bp3:left|bp3:right)/((bp3:component|bp3:memberPhysicalEntity)*) ?protein1 .
  ?protein1 rdf:type bp3:Protein .
  ?reaction ((^bp3:controlled/bp3:controller)|bp3:left|bp3:right)/((bp3:component|bp3:memberPhysicalEntity)*) ?protein2 .
  ?protein2 rdf:type bp3:Protein .
  FILTER (STR(?protein1) < STR(?protein2))
  
  OPTIONAL { ?protein1 bp3:displayName ?protein1Label . }
  OPTIONAL {
    ?protein1 bp3:xref [
      bp3:db "Reactome" ;
      bp3:id ?protein1ID
    ] .
  }
  
  OPTIONAL { ?protein2 bp3:displayName ?protein2Label . }
  OPTIONAL {
    ?protein2 bp3:xref [
      bp3:db "Reactome" ;
      bp3:id ?protein2ID
    ] .
  }
  
}
"""

sparql = SPARQLWrapper(endpoint_reactome)
sparql.setQuery(prefixes+query_neighbor_of)
df = save_for_cytoscape(sparql, prefixes, "../Results/ReactomeHomoSapiens93/08-NeighborOf.csv", format='csv')
preview_network_data(df)
print("--- %s seconds ---" % (time.time() - start_time))

11:34:26 INFO  Fuseki          :: [1] GET http://localhost:3030/reactome?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0APREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/93/48887%23%3E%0APREFIX+abstraction%3A%3Chttp%3A//abstraction/%23%3E%0A+%0ACONSTRUCT+%7B%0A+++%3Fprotein1+abstraction%3ANeighborOf+%3Fprotein2%0A%7D%0A

Saved 2841298 interactions to ../Results/ReactomeHomoSapiens93/08-NeighborOf.csv

Preview of network data (2841298 total interactions):

First 5 interactions:
                  Source             Interaction                 Target
0  reactome:Protein28490  abstraction:NeighborOf  reactome:Protein28671
1  reactome:Protein26395  abstraction:NeighborOf  reactome:Protein26407
2  reactome:Protein27742  abstraction:NeighborOf  reactome:Protein28569
3    reactome:Protein332  abstraction:NeighborOf    reactome:Protein422
4  reactome:Protein32309  abstraction:NeighborOf  reactome:Protein32525

Network statistics:
Number of unique nodes: 32104
Number of interactions: 2841298
Unique interaction types:
  - abstraction:NeighborOf
--- 281.09238266944885 seconds ---


#### 9 - Consumption controled by
The small molecule is consumed by a reaction that is controled by a protein 

Reaction test Reactome : R-HSA-2142688, Synthesis of 5-eicosatetranoic acids

In [7]:
start_time = time.time()
query_consumption_controled_by = """  
CONSTRUCT {
  ?reactant abstraction:ConsumptionControlledBy ?enzymeProt
}
WHERE {
  # Core reaction pattern
  ?reaction rdf:type bp3:BiochemicalReaction .
  ?reactant rdf:type bp3:SmallMolecule .
  ?product rdf:type bp3:SmallMolecule .
 
  ?reaction bp3:left ?reactant .
  ?reaction bp3:right ?product .
  ?catalysis bp3:controlled ?reaction .
  ?catalysis bp3:controller ?enzyme .
  ?enzyme ((bp3:component|bp3:memberPhysicalEntity)*) ?enzymeProt .
  ?enzymeProt rdf:type bp3:Protein .
}
"""

sparql = SPARQLWrapper(endpoint_reactome)
sparql.setQuery(prefixes+query_consumption_controled_by)
df = save_for_cytoscape(sparql, prefixes, "../Results/ReactomeHomoSapiens93/09-ConsumptionControledBy.csv", format='csv')
preview_network_data(df)
print("--- %s seconds ---" % (time.time() - start_time))

11:16:00 INFO  Fuseki          :: [2] GET http://localhost:3030/reactome?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0APREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/93/48887%23%3E%0APREFIX+abstraction%3A%3Chttp%3A//abstraction/%23%3E%0A++%0ACONSTRUCT+%7B%0A++%3Freactant+abstraction%3AConsumptionControlledBy+%3Fenzy

Saved 18839 interactions to ../Results/ReactomeHomoSapiens93/09-ConsumptionControledBy.csv

Preview of network data (18839 total interactions):

First 5 interactions:
                       Source                          Interaction  \
0    reactome:SmallMolecule22  abstraction:ConsumptionControlledBy   
1  reactome:SmallMolecule2362  abstraction:ConsumptionControlledBy   
2   reactome:SmallMolecule134  abstraction:ConsumptionControlledBy   
3    reactome:SmallMolecule11  abstraction:ConsumptionControlledBy   
4    reactome:SmallMolecule11  abstraction:ConsumptionControlledBy   

                  Target  
0  reactome:Protein20650  
1   reactome:Protein2172  
2  reactome:Protein19983  
3   reactome:Protein9239  
4   reactome:Protein7142  

Network statistics:
Number of unique nodes: 11633
Number of interactions: 18839
Unique interaction types:
  - abstraction:ConsumptionControlledBy
--- 0.7932698726654053 seconds ---


#### 10 - Controls Production 
The protein controls a reaction of which the small molecule is an output

Reaction test Reactome : R-HSA-2142688, Synthesis of 5-eicosatetranoic acids

In [8]:
start_time = time.time()
query_controls_production = """ 
CONSTRUCT {
  ?enzymeProt abstraction:ControlsProductionOf ?product
}
WHERE {
  # Core reaction pattern
  ?reaction rdf:type bp3:BiochemicalReaction .
 
  ?reaction bp3:left ?reactant .
  ?reaction bp3:right ?product .
  ?reactant rdf:type bp3:SmallMolecule .
  ?product rdf:type bp3:SmallMolecule .
  
  ?catalysis bp3:controlled ?reaction .
  ?catalysis rdf:type bp3:Catalysis .
  ?catalysis bp3:controller ?enzyme .
  ?enzyme ((bp3:component|bp3:memberPhysicalEntity)*) ?enzymeProt .
  ?enzymeProt rdf:type bp3:Protein .
}
"""

sparql = SPARQLWrapper(endpoint_reactome)
sparql.setQuery(prefixes+query_controls_production)
df = save_for_cytoscape(sparql, prefixes, "../Results/ReactomeHomoSapiens93/10-ControlsProduction.csv", format='csv')
preview_network_data(df)
print("--- %s seconds ---" % (time.time() - start_time))

11:17:07 INFO  Fuseki          :: [3] GET http://localhost:3030/reactome?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0APREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/93/48887%23%3E%0APREFIX+abstraction%3A%3Chttp%3A//abstraction/%23%3E%0A+%0ACONSTRUCT+%7B%0A++%3FenzymeProt+abstraction%3AControlsProductionOf+%3Fproduc

Saved 19598 interactions to ../Results/ReactomeHomoSapiens93/10-ControlsProduction.csv

Preview of network data (19598 total interactions):

First 5 interactions:
                  Source                       Interaction  \
0  reactome:Protein19899  abstraction:ControlsProductionOf   
1  reactome:Protein25403  abstraction:ControlsProductionOf   
2   reactome:Protein4411  abstraction:ControlsProductionOf   
3   reactome:Protein5881  abstraction:ControlsProductionOf   
4  reactome:Protein20091  abstraction:ControlsProductionOf   

                       Target  
0  reactome:SmallMolecule2966  
1    reactome:SmallMolecule50  
2   reactome:SmallMolecule101  
3    reactome:SmallMolecule12  
4  reactome:SmallMolecule1889  

Network statistics:
Number of unique nodes: 11539
Number of interactions: 19598
Unique interaction types:
  - abstraction:ControlsProductionOf
--- 0.9496521949768066 seconds ---


#### 11 - Controls Transport Of Chemical
The protein controls a reaction that changes the cellular location of the small molecule

Reaction test : R-HSA-70635, Urea Cycle

In [10]:
start_time = time.time()
query_controls_transport_of_chemical = """ 
CONSTRUCT {
  ?enzymeProt abstraction:ControlsTransportOfChemical ?SmallMolecule1
}
WHERE {
  # Core reaction pattern
  ?SmallMolecule1 rdf:type bp3:SmallMolecule ;
        	bp3:cellularLocation ?cellularLocVocab1 .
  ?SmallMolecule2 rdf:type bp3:SmallMolecule ;
        	bp3:cellularLocation ?cellularLocVocab2 .
 
  ?SmallMolecule1 bp3:entityReference ?SmallMolecule1Ref .
  ?SmallMolecule2 bp3:entityReference ?SmallMolecule2Ref .
  ?cellularLocVocab1 bp3:term ?location1 .
  ?cellularLocVocab2 bp3:term ?location2 .

  ?reaction rdf:type bp3:BiochemicalReaction ;
                   	bp3:left ?SmallMolecule1 ;
                   	bp3:right ?SmallMolecule2 .

  ?catalysis bp3:controlled ?reaction .
  ?catalysis rdf:type bp3:Catalysis .
  ?catalysis bp3:controller ?enzyme .
  ?enzyme ((bp3:component|bp3:memberPhysicalEntity)*) ?enzymeProt .
  ?enzymeProt rdf:type bp3:Protein .
}
"""

sparql = SPARQLWrapper(endpoint_reactome)
sparql.setQuery(prefixes+query_controls_transport_of_chemical)
df = save_for_cytoscape(sparql, prefixes, "../Results/ReactomeHomoSapiens93/11-ControlsTransportOfChemical.csv", format='csv')
preview_network_data(df)
print("--- %s seconds ---" % (time.time() - start_time))

11:20:38 INFO  Fuseki          :: [5] GET http://localhost:3030/reactome?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0APREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/93/48887%23%3E%0APREFIX+abstraction%3A%3Chttp%3A//abstraction/%23%3E%0A+%0ACONSTRUCT+%7B%0A++%3FenzymeProt+abstraction%3AControlsTransportOfChemical+%3

Saved 17784 interactions to ../Results/ReactomeHomoSapiens93/11-ControlsTransportOfChemical.csv

Preview of network data (17784 total interactions):

First 5 interactions:
                  Source                              Interaction  \
0  reactome:Protein20041  abstraction:ControlsTransportOfChemical   
1  reactome:Protein25014  abstraction:ControlsTransportOfChemical   
2   reactome:Protein3052  abstraction:ControlsTransportOfChemical   
3  reactome:Protein21762  abstraction:ControlsTransportOfChemical   
4  reactome:Protein23720  abstraction:ControlsTransportOfChemical   

                       Target  
0  reactome:SmallMolecule1330  
1     reactome:SmallMolecule1  
2     reactome:SmallMolecule6  
3  reactome:SmallMolecule4632  
4     reactome:SmallMolecule1  

Network statistics:
Number of unique nodes: 11090
Number of interactions: 17784
Unique interaction types:
  - abstraction:ControlsTransportOfChemical
--- 0.9488685131072998 seconds ---


#### 12 - Chemical Affects
A small molecule has an effect on the protein state

Reaction test Reactome : R-HSA-202437,  Phosphorylation of CARMA1 

In [11]:
start_time = time.time()
query_chemical_affects = """  
CONSTRUCT {
  ?catalyzerChemical abstraction:ChemicalAffects ?protLeft
}
WHERE {
  # Core reaction pattern
  ?reaction rdf:type/(rdfs:subClassOf*) bp3:BiochemicalReaction .
  ?reaction bp3:left/((bp3:component|bp3:memberPhysicalEntity)*) ?protLeft .
  ?protLeft rdf:type bp3:Protein .
  ?protLeft bp3:entityReference ?leftProteinRef .
  ?reaction bp3:right/((bp3:component|bp3:memberPhysicalEntity)*) ?protRight .
  ?protRight rdf:type bp3:Protein .
  ?protRight bp3:entityReference ?rightProteinRef . 
  
  ?catalysis bp3:controlled ?reaction .
  ?catalysis rdf:type bp3:Catalysis .
  ?catalysis bp3:controller ?catalyzer .
  ?catalyzer ((bp3:component|bp3:memberPhysicalEntity)*) ?catalyzerChemical .
  ?catalyzerChemical rdf:type bp3:SmallMolecule .

  # Complete modification check
  ?protRight bp3:feature ?feature .
  ?feature (rdf:type/rdfs:subClassOf*) bp3:EntityFeature .
 
  # Ensure left and right proteins are the same
  FILTER(?leftProteinRef = ?rightProteinRef)
}
"""

sparql = SPARQLWrapper(endpoint_reactome)
sparql.setQuery(prefixes+query_chemical_affects)
df = save_for_cytoscape(sparql, prefixes, "../Results/ReactomeHomoSapiens93/12-ChemicalAffects.csv", format='csv')
preview_network_data(df)
print("--- %s seconds ---" % (time.time() - start_time))

11:24:24 INFO  Fuseki          :: [6] GET http://localhost:3030/reactome?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0APREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/93/48887%23%3E%0APREFIX+abstraction%3A%3Chttp%3A//abstraction/%23%3E%0A++%0ACONSTRUCT+%7B%0A++%3FcatalyzerChemical+abstraction%3AChemicalAffects+%3Fpro

Saved 32434 interactions to ../Results/ReactomeHomoSapiens93/12-ChemicalAffects.csv

Preview of network data (32434 total interactions):

First 5 interactions:
                       Source                  Interaction  \
0   reactome:SmallMolecule732  abstraction:ChemicalAffects   
1  reactome:SmallMolecule1065  abstraction:ChemicalAffects   
2    reactome:SmallMolecule97  abstraction:ChemicalAffects   
3  reactome:SmallMolecule2137  abstraction:ChemicalAffects   
4  reactome:SmallMolecule2166  abstraction:ChemicalAffects   

                  Target  
0  reactome:Protein16544  
1  reactome:Protein14140  
2   reactome:Protein2001  
3  reactome:Protein14037  
4   reactome:Protein6995  

Network statistics:
Number of unique nodes: 4051
Number of interactions: 32434
Unique interaction types:
  - abstraction:ChemicalAffects
--- 27.540232181549072 seconds ---


#### 13 - Reacts With
Small molecules are input to a biochemical reaction 

Reaction test Reactome : R-HSA-2142688, Synthesis of 5-eicosatetranoic acids

In [12]:
start_time = time.time()
query_reacts_with = """ 
CONSTRUCT {
  ?smallMolecule1 abstraction:reactsWith ?smallMolecule2 .
}
WHERE {
  ?smallMolecule1 rdf:type bp3:SmallMolecule .
  ?smallMolecule2 rdf:type bp3:SmallMolecule .
 
  ?reaction bp3:left ?smallMolecule1 .
  ?reaction bp3:left ?smallMolecule2 .
  ?reaction rdf:type bp3:BiochemicalReaction .
 
  FILTER (STR(?smallMolecule1) < STR(?smallMolecule2))
}
"""

sparql = SPARQLWrapper(endpoint_reactome)
sparql.setQuery(prefixes+query_reacts_with)
df = save_for_cytoscape(sparql, prefixes, "../Results/ReactomeHomoSapiens93/13-ReactsWith.csv", format='csv')
preview_network_data(df)
print("--- %s seconds ---" % (time.time() - start_time))

11:25:44 INFO  Fuseki          :: [7] GET http://localhost:3030/reactome?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0APREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/93/48887%23%3E%0APREFIX+abstraction%3A%3Chttp%3A//abstraction/%23%3E%0A+%0ACONSTRUCT+%7B%0A++%3FsmallMolecule1+abstraction%3AreactsWith+%3FsmallMolecul

Saved 2497 interactions to ../Results/ReactomeHomoSapiens93/13-ReactsWith.csv

Preview of network data (2497 total interactions):

First 5 interactions:
                       Source             Interaction  \
0  reactome:SmallMolecule1046  abstraction:reactsWith   
1   reactome:SmallMolecule360  abstraction:reactsWith   
2  reactome:SmallMolecule2977  abstraction:reactsWith   
3  reactome:SmallMolecule2355  abstraction:reactsWith   
4  reactome:SmallMolecule1059  abstraction:reactsWith   

                       Target  
0  reactome:SmallMolecule2783  
1  reactome:SmallMolecule5353  
2    reactome:SmallMolecule30  
3  reactome:SmallMolecule4324  
4  reactome:SmallMolecule2793  

Network statistics:
Number of unique nodes: 1657
Number of interactions: 2497
Unique interaction types:
  - abstraction:reactsWith
--- 26.436919689178467 seconds ---


11:26:10 INFO  Fuseki          :: [7] 200 OK (26.384 s)


#### 14 - Used To Produce
A reaction consumes a small molecule to produce another small molecule

In [13]:
start_time = time.time()
query_used_to_produce = """ 
CONSTRUCT {
  ?smallMolecule1 abstraction:UsedToProduce ?smallMolecule2 .
}
WHERE {
  ?smallMolecule1 rdf:type bp3:SmallMolecule .
  ?smallMolecule2 rdf:type bp3:SmallMolecule .
 
  ?reaction bp3:left ?smallMolecule1 .
  ?reaction bp3:right ?smallMolecule2 .
  ?reaction rdf:type bp3:BiochemicalReaction .
 
  FILTER (?smallMolecule1 != ?smallMolecule2)
}
"""

sparql = SPARQLWrapper(endpoint_reactome)
sparql.setQuery(prefixes+query_used_to_produce)
df = save_for_cytoscape(sparql, prefixes, "../Results/ReactomeHomoSapiens93/14-UsedToProduce.csv", format='csv')
preview_network_data(df)
print("--- %s seconds ---" % (time.time() - start_time))

11:26:40 INFO  Fuseki          :: [8] GET http://localhost:3030/reactome?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0APREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/93/48887%23%3E%0APREFIX+abstraction%3A%3Chttp%3A//abstraction/%23%3E%0A+%0ACONSTRUCT+%7B%0A++%3FsmallMolecule1+abstraction%3AUsedToProduce+%3FsmallMole

Saved 7885 interactions to ../Results/ReactomeHomoSapiens93/14-UsedToProduce.csv

Preview of network data (7885 total interactions):

First 5 interactions:
                       Source                Interaction  \
0  reactome:SmallMolecule1880  abstraction:UsedToProduce   
1  reactome:SmallMolecule2547  abstraction:UsedToProduce   
2  reactome:SmallMolecule2741  abstraction:UsedToProduce   
3  reactome:SmallMolecule2232  abstraction:UsedToProduce   
4    reactome:SmallMolecule17  abstraction:UsedToProduce   

                       Target  
0  reactome:SmallMolecule2973  
1    reactome:SmallMolecule14  
2  reactome:SmallMolecule2761  
3   reactome:SmallMolecule528  
4  reactome:SmallMolecule2688  

Network statistics:
Number of unique nodes: 2873
Number of interactions: 7885
Unique interaction types:
  - abstraction:UsedToProduce
--- 39.23868989944458 seconds ---


11:27:19 INFO  Fuseki          :: [8] 200 OK (39.103 s)


In [14]:
# end process
process.kill()
time.sleep(60)